# Whereabouts Plans Template
This notebook will demonstrate how to create a whereabouts plan automatically.

<div style="text-align:center"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/Downtown_Austin_%28southward_view%2C_from_the_Capitol_Grounds_on_11th_street%29_%2823_November_2006%29.jpg/640px-Downtown_Austin_%28southward_view%2C_from_the_Capitol_Grounds_on_11th_street%29_%2823_November_2006%29.jpg" /></div>

## Introduction
The purpose of this notebook is to create a Street and Bridge Work Order plans based on segment IDs and additional comments on long line. Markings feature layers are published in the City of Austin ArcGIS Portal page available for public view as well. 

The data should already be available in the folder path as an excel spreadsheet. If the spreadsheet exists, a map document will be configured for spreadsheet use.

## Imports
The packages used for this project are:
- [pandas](https://pandas.pydata.org/) to create dataframe of extracted table and transform the data
- [pathlib](https://docs.python.org/3/library/pathlib.html) to find path to excel document if it exists
- [PyPDF2](https://pythonhosted.org/PyPDF2/) to merge PDFs for cover and pages in order
- [archook](https://github.com/JamesRamm/archook) to search for arcgis and makes arcpy available to python 
- [arcpy](https://pro.arcgis.com/en/pro-app/arcpy/get-started/what-is-arcpy-.htm) to create whereabouts markings plans using ESRI ArcMap Desktop software

In [1]:
import pandas as pd
from pathlib import Path
from PyPDF2 import PdfFileMerger

import archook 
archook.get_arcpy()
import arcpy

import datetime
import os 

## Constants

The date by month and day constant will determine the file pdf name to use as a dataframe. Folder path will determine where the plans will be created depending on the year. This is set to the top for the purpose of changing these constants as needed.

<i>The table below explains the purpose of each constant.</i>

| Constant | Description   |
|:--------:|----|
| <b>MONTH, DAY, YEAR</b> |Date used to find PDF in month-day format and file path based on year|
|<b>FOLDER</b>      |File directory used to import SBO whereabouts reports from email|
|<b>FILE_NAME</b>   |File directory name used to extact SBO whereabouts reports from file|
|<b>MXD</b>   |Map document used to create whereabouts plans template|

In [2]:
%store -r MONTH
%store -r DAY
%store -r YEAR
%store -r FOLDER
%store -r FILE_NAME
MXD = arcpy.mapping.MapDocument(r"C:\Users\Govs\Projects\Whereabouts\Whereabouts_Template.mxd")
PLAN_PATH = r"G:\ATD\Signs_and_Markings\MARKINGS\Whereabouts WORK ORDERS\{}".format(YEAR)

## Methods
These functions will be used to extract and transform the data into a feasible format.

<i>The table below explains the purpose of each:</i>

| Method | Description   |
|:--------:|----|
|<b>df_text</b> |Changes element text to the pandas dataframe|
|<b>cover_aerial</b> |Zooms to selected features and extent for the dataframe|

In [19]:
# Formats element text to match pandas dataframe for cover
def df_text(r,row1,location,wg):
    date = datetime.datetime.now()
    for e in arcpy.mapping.ListLayoutElements(MXD,'TEXT_ELEMENT'):
            if e.name == 'SPECIFICATIONS':
                e.text = str(r["SPECIFICATIONS"])
            elif e.name == 'CREATED DATE':
                e.text =  "{}/{}/{}".format(date.month,date.day,date.year)
            elif e.name == 'REQUESTOR ID':
                e.text = str(r["Location ID"])
            elif e.name == 'row1':
                e.text = row1
            elif e.name == 'LOCATION':
                e.text = location
            elif e.name == 'author':
                e.text = 'Susanne Gov'
            elif e.name == 'WORK GROUPS':
                e.text = wg
            elif e.name == 'PAGE':
                e.text = str(r['PAGE'])
                
# Refreshes dataframe map aerial                
def aerial(layer,scale,select):
    dataframe = arcpy.mapping.ListDataFrames(MXD,"Layers")[0]
    dataframe.zoomToSelectedFeatures()
    dataframe.extent = layer.getSelectedExtent()
    dataframe.scale = dataframe.scale * scale
    if select == True:
        arcpy.SelectLayerByAttribute_management(layer,"CLEAR_SELECTION")
    arcpy.RefreshActiveView()

# Goes through mxd layers and allows selection/deselection based on query
# On to do list: do definition query instead for markings
def select_layers(q,dq,option):
    for l in arcpy.mapping.ListLayers(MXD,'TRANSPORTATION.markings*'):
        arcpy.SelectLayerByAttribute_management(l,option,q)
        l.definitionQuery = dq

## Import Datasets

In [17]:
if Path(FILE_NAME + '.xlsx').exists():
    df = pd.read_excel(FILE_NAME + '.xlsx','Cover', index_col=0).reset_index()
    df1 = pd.read_excel(FILE_NAME + '.xlsx','Pages', index_col=0).reset_index()
    display(df1)

,PAGE,Location ID,SEGMENT_ID,BLOCK,CROSSWALK,STOP_LINE,Bicyclist symbol,Bike arrow,Diagonal crosshatch,Left arrow,Only word,Right arrow,SPECIFICATIONS
0,9,62870,2026275,4901,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,"Install 3 bike arrow, 4 diagonal crosshatch"
1,2,62870,2038344,4310,NaN,NaN,1.0,4.0,NaN,NaN,NaN,NaN,"Install 1 bicyclist symbol, 4 bike arrow"
2,8,62870,2038345,4900,NaN,NaN,2.0,1.0,NaN,NaN,1.0,1.0,"Install 2 bicyclist symbol, 1 bike arrow, 1 on..."
3,10,74361,2040261,8401,2.0,2.0,NaN,NaN,13.0,4.0,1.0,2.0,"Install 2 crosswalk, 2 stop_line, 13 diagonal ..."
4,3,62870,2043055,4311,NaN,NaN,1.0,4.0,NaN,NaN,NaN,NaN,"Install 1 bicyclist symbol, 4 bike arrow"
5,4,62870,3259330,4600,1.0,1.0,1.0,3.0,5.0,NaN,NaN,NaN,"Install 1 crosswalk, 1 stop_line, 1 bicyclist ..."
6,6,62870,3259331,4718,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,"Install 1 bicyclist symbol, 2 bike arrow"
7,5,62870,3259334,4601,NaN,NaN,1.0,2.0,1.0,NaN,NaN,NaN,"Install 1 bicyclist symbol, 2 bike arrow, 1 di..."
8,7,62870,3259335,4719,NaN,1.0,1.0,2.0,NaN,NaN,NaN,NaN,"Install 1 stop_line, 1 bicyclist symbol, 2 bik..."


# Creating the Map

In [18]:
sde_path = r"Database Connections\GISDM.sde"
pdf_file = PLAN_PATH + "\\ATD_Whereabouts_"

# Create Database connection 
arcpy.env.workspace = sde_path
if arcpy.Exists(sde_path) == False:
    arcpy.CreateDatabaseConnection_management("Database Connections","GISDM.sde", "ORACLE", 
                                              "sde:oracle11g:gisdm", "DATABASE_AUTH")

# Create layers
layer = arcpy.mapping.ListLayers(MXD,"TRANSPORTATION.street_segment")[0]
sl = arcpy.mapping.ListLayers(MXD,'TRANSPORTATION.markings_short_line')[0]
sp = arcpy.mapping.ListLayers(MXD,'TRANSPORTATION.markings_specialty_point')[0]

## Setting up Cover Document
A database connection to GISDM is needed to access the markings asset layers and the markings assets layers. We can open up a map document of the cover work orders and another map document of the pages work orders.

In [27]:
for index,row in df.iterrows():
    q = 'SEGMENT_ID IN({})'.format(str(str(row["Segment IDs"]).split(','))[1:-1])
    select_layers(q,q,'NEW_SELECTION')
    arcpy.SelectLayerByAttribute_management(layer,"NEW_SELECTION",q) 
    df_text(row,'LOCATION',"{} from {} to {}".format(row["Street"],row["From"],row["To"]),str(row['WORK GROUPS']))
    select_layers(q,q,'CLEAR_SELECTION')
    aerial(layer,1,False)
    arcpy.mapping.ExportToPDF(MXD, pdf_file + "Cover_{}.pdf".format(row["Location ID"]))
    select_layers(q,'',"CLEAR_SELECTION")

## Setting up Pages

In [28]:
for index,row in df1.iterrows():
    q = 'SEGMENT_ID = {}'.format(int(row["SEGMENT_ID"]))
    select_layers(q,q,"NEW_SELECTION")
    arcpy.SelectLayerByAttribute_management(layer,"NEW_SELECTION",q) 
    df_text(row,'SEGMENT ID',str(row['SEGMENT_ID']),'Markings')
    select_layers(q,q,'CLEAR_SELECTION')
    aerial(layer,1.05,True)
    arcpy.mapping.ExportToPDF(MXD, pdf_file + 'Page_{}_{}_{}.pdf'.format(row["Location ID"],row['SEGMENT_ID'],row['PAGE']))
    select_layers(q,'',"CLEAR_SELECTION")

## Merging Cover and Pages

In [9]:
# merge these in order of pdf
for i,r in df.iterrows():
    merger = PdfFileMerger()
    pdf = "{}{}_{}.pdf".format(a,'Cover',r['Location ID'])
    merger.append(pdf)
    for index,row in df1.iterrows():
        if r['Location ID'] == row['Location ID']:
            pdf = "{}{}_{}_{}.pdf".format(a,'Page',row['Location ID'],row['SEGMENT_ID'])
            merger.append(pdf)
    merger.write(a + "{}_Final.pdf".format(r['Location ID']))
    merger.close()

Complete!

## Kill ArcMap Desktop

In [24]:
import os  
os.system("TASKKILL /F /IM ArcMap.exe")  

128